In [0]:
#          _____                _____                    _____                    _____                    _____                    _____          
#         /\    \              /\    \                  /\    \                  /\    \                  /\    \                  /\    \         
#        /::\    \            /::\    \                /::\    \                /::\    \                /::\    \                /::\    \        
#       /::::\    \           \:::\    \              /::::\    \              /::::\    \              /::::\    \               \:::\    \       
#      /::::::\    \           \:::\    \            /::::::\    \            /::::::\    \            /::::::\    \               \:::\    \      
#     /:::/\:::\    \           \:::\    \          /:::/\:::\    \          /:::/\:::\    \          /:::/\:::\    \               \:::\    \     
#    /:::/__\:::\    \           \:::\    \        /:::/__\:::\    \        /:::/__\:::\    \        /:::/__\:::\    \               \:::\    \    
#    \:::\   \:::\    \          /::::\    \      /::::\   \:::\    \      /::::\   \:::\    \      /::::\   \:::\    \              /::::\    \   
#  ___\:::\   \:::\    \        /::::::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    ____    /::::::\    \  
# /\   \:::\   \:::\    \      /:::/\:::\    \  /:::/\:::\   \:::\    \  /:::/\:::\   \:::\____\  /:::/\:::\   \:::\    \  /\   \  /:::/\:::\    \ 
#/::\   \:::\   \:::\____\    /:::/  \:::\____\/:::/  \:::\   \:::\____\/:::/  \:::\   \:::|    |/:::/  \:::\   \:::\____\/::\   \/:::/  \:::\____\
#\:::\   \:::\   \::/    /   /:::/    \::/    /\::/    \:::\  /:::/    /\::/   |::::\  /:::|____|\::/    \:::\  /:::/    /\:::\  /:::/    \::/    /
# \:::\   \:::\   \/____/   /:::/    / \/____/  \/____/ \:::\/:::/    /  \/____|:::::\/:::/    /  \/____/ \:::\/:::/    /  \:::\/:::/    / \/____/ 
#  \:::\   \:::\    \      /:::/    /                    \::::::/    /         |:::::::::/    /            \::::::/    /    \::::::/    /          
#   \:::\   \:::\____\    /:::/    /                      \::::/    /          |::|\::::/    /              \::::/    /      \::::/____/           
#    \:::\  /:::/    /    \::/    /                       /:::/    /           |::| \::/____/               /:::/    /        \:::\    \           
#     \:::\/:::/    /      \/____/                       /:::/    /            |::|  ~|                    /:::/    /          \:::\    \          
#      \::::::/    /                                    /:::/    /             |::|   |                   /:::/    /            \:::\    \         
#       \::::/    /                                    /:::/    /              \::|   |                  /:::/    /              \:::\____\        
#        \::/    /                                     \::/    /                \:|   |                  \::/    /                \::/    /        
#         \/____/                                       \/____/                  \|___|                   \/____/                  \/____/    

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# install dependancies, takes around 45 seconds

Rendering Dependancies



In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

CarRacing Dependancies

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install gym[box2d] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.4.0)


# Imports and Helper functions


In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#4/0AEXSDU83MzetlCnwLKFRv3W43C5G-WhIepNiyNchv8Gd8YSEqmkue4

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/DQN/'

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17373878368524217724
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3664254692241519784
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15640724757568788729
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15701463552
locality {
  bus_id: 1
  links {
  }
}
incarnation: 942780294564178408
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


Car Racing DQN

In [0]:

import numpy as np
import gym
import random
from collections import deque
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import pickle
import time

#Preprocessing
def gray_scale(img):
    return np.dot(img, [0.299, 0.587, 0.144])

def crop(img, top=6, bottom=6, left=6, right=6):
    h, w = img.shape
    return img[top : h - bottom, left : w - right]

def preprocess(img):
    return crop(gray_scale(img)).reshape(1, 84, 84, 1)

class ReplayMemory(object):
    def __init__(self, max_size):
        self.max_size = max_size
        self.cntr = 0
        self.mem = deque(maxlen = self.max_size)

    def store(self, phi_St, action, reward, next_phi_St, done):
        self.mem.append((phi_St, action, reward, next_phi_St, done))
        self.cntr += 1

    def sample(self, batch_size):
        batch_size = min(batch_size, self.cntr)
        return batch_size, random.sample(self.mem, batch_size)


class DQNAgent(object):
    def __init__(self):
        self.input_shape = (84, 84, 1)
        self.action_count = 3
        self.epsilon_end = 0.01
        self.epsilon_step = 0.996 #0.0009 #0.0001
        self.no_of_episodes = 2 #no of episodes
        self.replay_mem_max_size = 100000 #1000000 #replay memory capacity
        self.batch_size = 32 #16 #do trial-error #replay batch size
        self.skip_frames = 4
        self.gamma = 0.99 #discount factor
        self.C = 5000 #10000
        self.l_rate = 0.0005 #0.00025
        self.discrete_action_ids = [i for i in range(self.action_count)]
        self.discrete_actions = dict([(0, [0.0, 1.0, 0.0]), #UP
                            (1, [-1.0, 0.0, 0.0]), #LEFT
                            (2, [1.0, 0.0, 0.0]), #RIGHT
                           # (3, [0.0, 0.0, 0.0])  #DO NOTHING
                           # (3, [0.0, 0.0, 1.0])  #BRAKE
                            ])
        self.epi_score = 0
        self.total_score = 0
        self.loss = 0
        self.skip_cntr = 0
        self.log_file = open(base_dir + "data/dqn_log.txt", "a+")
        self.replay_mem_file = base_dir + "data/replay_mem_data.file"
        self.epsilon_file = base_dir + "data/epsilon.file"
        self.latest_weights_file = base_dir + "data/latest_weights.h5"  #My Drive/Colab Notebooks/DQN/data/

        try:
            self.replay_mem = pickle.load(open(self.replay_mem_file, "rb"))
            print("Replay Memory loaded. Length: ", len(self.replay_mem.mem))
        except Exception as e:
            self.replay_mem = ReplayMemory(self.replay_mem_max_size)
            print("Error in loading Replay Memory. ", e)

        try:
            self.epsilon = pickle.load(open(self.epsilon_file, "rb"))
            print("Epsilon loaded. Value: ", self.epsilon)
        except Exception as e:
            self.epsilon = 1
            print("Error in loading Epsilon. ", e)

    def create_model(self):
        self.model = tfk.models.Sequential([
        tfkl.Conv2D(16, (8, 8), strides = 4, activation = 'relu', input_shape=self.input_shape), #elu or relu
        tfkl.Conv2D(32, (4, 4), strides = 2, activation = 'relu'),
        tfkl.Conv2D(32, (3, 3), strides = 1, activation = 'relu'),
        tfkl.Flatten(),
        tfkl.Dense(256, activation='relu'),
        tfkl.Dense(self.action_count, activation='softmax')
        ])
        self.model.compile(optimizer=tfk.optimizers.RMSprop(learning_rate=self.l_rate, momentum=0.95), loss='mse')

        try:
            self.model.load_weights(self.latest_weights_file)
            print("Latest weights loaded.")
        except Exception as e:
            print("Error in loading Latest weights. ", e)
        self.new_weights = self.old_weights = self.model.get_weights()

    def take_skip_action(self):
        if self.skip_cntr % self.skip_frames != 0:
            next_x, reward, done, info = env.step(self.discrete_actions[self.skip_action])
            self.epi_score += reward
            self.skip_cntr += 1
            skipped = True
        else:
            next_x = None
            reward = 0
            done = False
            info = None
            self.skip_cntr = 0
            skipped = False
        return next_x, reward, done, info, skipped

    def take_e_greedy_action(self, current_state):
        if random.uniform(0, 1) < self.epsilon:
            action = np.random.choice(self.discrete_action_ids)
        else:
            #Q.set_weights(new_weights)
            actions = self.model.predict(current_state)
            action = np.argmax(actions)
            print(actions)
            print(action)
        self.skip_action = action
        next_x, reward, done, info = env.step(self.discrete_actions[action])
        self.epi_score += reward
        return preprocess(next_x), reward, done, info
    
    def take_predicted_action(self, current_state):
        actions = self.model.predict(current_state)
        action = np.argmax(actions)
        next_x, reward, done, info = env.step(self.discrete_actions[action])
        self.epi_score += reward
        return preprocess(next_x), action, reward, done, info

    def update_epsilon(self):
        #self.epsilon -= self.epsilon_step
        if self.epsilon > self.epsilon_end:
            self.epsilon = self.epsilon * self.epsilon_step 
        else:
            self.epsilon = self.epsilon_end        

    def store_transition(self, phi_St, action, reward, next_phi_St, done):
        self.replay_mem.store(np.squeeze(phi_St, axis=0), action, reward, np.squeeze(next_phi_St, axis=0), float(done))

    def sample_transition(self):
        batch_size, samples = self.replay_mem.sample(self.batch_size)
        phi_Sts, actions, rewards, next_phi_Sts, dones = zip(*samples)
        phi_Sts = tf.stack(phi_Sts)
        actions = tf.stack(actions)
        rewards = tf.stack(rewards)
        next_phi_Sts = tf.stack(next_phi_Sts)
        dones = tf.stack(dones)
        return batch_size, phi_Sts, actions, rewards, next_phi_Sts, dones

    def optimize_loss(self, batch_size, phi_Sts, actions, rewards, next_phi_Sts, dones):
        self.model.set_weights(self.old_weights)
        q_vals = self.model.predict(next_phi_Sts)
        q_vals = np.max(q_vals, axis=1)
        batch_list = np.arange(batch_size, dtype=np.int32)
        self.model.set_weights(self.new_weights)
        target_q_vals = self.model.predict(phi_Sts)
        target_q_vals[batch_list, actions] = rewards + (1 - dones) * self.gamma * q_vals
        self.old_weights = self.new_weights
        hist = self.model.fit(phi_Sts, target_q_vals, verbose=0)
        self.new_weights = self.model.get_weights()
        self.loss = hist.history['loss']
        #val_loss

    def reset_target_params(self, epochs):
        if epochs % self.C == 0:
            self.old_weights = self.new_weights

    def end_episode(self):
        self.total_score += self.epi_score
        self.epi_score = 0

    def print_status(self, episode, elapsed_time):
        str = "\nepisode: {}  score: {:.5f}  loss: {:.5f}  epsilon: {:.3f}  {:.2f}s {}"
        str = str.format(episode, self.epi_score, np.sum(self.loss), self.epsilon, \
                         elapsed_time, time.strftime("%Y-%m-%d %H:%M:%S"))
        self.log_file.write(str)
        print(str)

    def close(self):
        self.log_file.close()

    def save_partial_status(self, episode):
        if episode % 5 == 0 or episode == self.no_of_episodes:
            self.model.save_weights(self.latest_weights_file)
            with open(self.replay_mem_file, "wb") as f:
                pickle.dump(self.replay_mem, f, pickle.HIGHEST_PROTOCOL)
            with open(self.epsilon_file, "wb") as f:
                pickle.dump(self.epsilon, f, pickle.HIGHEST_PROTOCOL)

        if episode % 10 == 0:
            timestr = time.strftime("%Y%m%d_%H%M%S")
            self.model.save_weights(base_dir + "data/weights_{}_{}.h5".format(episode, timestr)) 


TRAINING

In [18]:
agent = DQNAgent()
agent.create_model()

action = np.random.choice(agent.discrete_action_ids)

epochs = 0
env = gym.make ('CarRacing-v0') 
#env = wrap_env(gym.make('CarRacing-v0'))
#print(env.observation_space, env.action_space)

for episode in range(1, agent.no_of_episodes+1):
    start_time = time.time()
    #epsilon = epsilon/2
    done = False
    phi_St = preprocess(env.reset())

    while not done:
        epochs += 1
        next_x, reward, done, info, skipped = agent.take_skip_action()
        if skipped:
            continue

        next_phi_St, reward, done, info = agent.take_e_greedy_action(phi_St)

        agent.store_transition(phi_St, action, reward, next_phi_St, done)
        batch_size, phi_Sts, actions, rewards, next_phi_Sts, dones = agent.sample_transition()

        agent.optimize_loss(batch_size, phi_Sts, actions, rewards, next_phi_Sts, dones)

        agent.reset_target_params(epochs)
        #agent.print_status(1, 0)
        #print(f"{epochs}")

    agent.update_epsilon()
    agent.print_status(episode, time.time() - start_time)
    agent.save_partial_status(episode)
    agent.end_episode()

print(f"Total Score: {agent.total_score}")
env.close()
agent.close()

Replay Memory loaded. Length:  2888
Epsilon loaded. Value:  0.984095744256
Latest weights loaded.
Track generation: 1095..1376 -> 281-tiles track
retry to generate track (normal if there are not many instances of this message)
Track generation: 1151..1443 -> 292-tiles track
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1

episode: 1  score: -128.60790  loss: 0.10153  epsilon: 0.980  417.35s 2020-05-22 08:30:56
Track generation: 1100..1379 -> 279-tiles track
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1
[[0. 1. 0.]]
1

episode: 2  score: -82.06043  loss: 0.09341  epsilon: 0.976  255.19s 2020-05-22 08:35:11
Total Score: -210.6683354347463

EVALUATE

In [0]:
agent = DQNAgent()
agent.create_model()

epochs = 0
env = wrap_env(gym.make('CarRacing-v0')) #gym.make ('CarRacing-v0') 

done = False
phi_St = preprocess(env.reset())
#env.render()

while not done:
    start_time = time.time()
    epochs += 1
    phi_St, action, reward, done, info = agent.take_predicted_action(phi_St)
    print(action)
    #env.render()
    #agent.print_status(1, 0)
    #print(f"{epochs}")    
print("Reward: ", agent.epi_score, " Time: ", time.time() - start_time)  
agent.end_episode()

print(f"Total Score: {agent.total_score}")
env.close()
agent.close()

In [0]:
show_video()